In [1]:
%%bash
pip install -qU gigachain==0.2.6 gigachain_community==0.2.6 gigachain-cli==0.0.25 langgraph==0.2.19 langchain-community==0.2.16 langchain-openai==0.1.23  python-pptx==0.6.23 python-pptx-interface==0.0.12 python-dotenv==1.0.1 duckduckgo-search==6.2.4 faiss-cpu

In [1]:
import getpass
import os
import requests
import json
import time

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings

from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool

import faiss
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from pydantic import BaseModel

In [ ]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('.env')

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [13]:
import os
import sys

class SimpleChatBot:
    def __init__(self, llm: GigaChat, memory_type: str, memory_window_size: int):
        self.llm = llm
        self.memory_type = memory_type
        self.window_size = memory_window_size
        self.memory = self._get_memory(memory_type, memory_window_size)
        self.conversation = ConversationChain(
            llm=self.llm,
            verbose=True,
            memory=self.memory
        )

    def _respond(self, user_input: str) -> str:
        response = self.conversation.predict(input=user_input)
        return response
    
    def print_memory(self) -> None:
        if self.memory:
            memory_content = self.memory.load_memory_variables({})
            print("Current memory state:")
            print(memory_content)
        else:
            print("No memory initialized or available.")

    def run(self) -> None:
        print("This is a simple chat bot:")
        try:
            while True:
                user_input = input('User: ')
                if user_input.lower() in {"exit", "quit"}:
                    print("Exiting chat...")
                    break
                response = self._respond(user_input)
                print(f'Bot: {response}')
                sys.stdout.flush()
        except KeyboardInterrupt:
            print("\nExiting chat due to keyboard interruption.")

    def _get_memory(self, memory_type: str, window_size: int):
        if memory_type == "summary":
            return ConversationSummaryMemory(llm=self.llm)
        elif memory_type == "window":
            return ConversationBufferWindowMemory(k=window_size)
        elif memory_type == "simple":
            return ConversationBufferMemory()
        else:
            raise ValueError(f"Unsupported memory type '{memory_type}'. Supported types are: 'summary', 'window', 'simple'.")


Now let's check how it works with each type of memory

In [14]:
giga_key = os.environ.get("SB_AUTH_DATA")
print(giga_key)
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary', memory_window_size=2)
chat.run()
chat.print_memory()

OWQzZDQxZDMtNmJiMS00ZjEyLWJiZTUtNTAwNTE5ZDNjNjI4OjYxNmE4YWQ1LWIzMjMtNDNlMi05YWM3LTMyYTA4YWExNzM5Mw==
This is a simple chat bot:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi! Can you solve following equation: 5 +1 
AI:

> Finished chain.
Bot: Sure! The solution to the equation 5 + 1 is 6.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human asked the AI to solve an equation, and the AI answered that the solution is 6.
Human: Nic

In [15]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: where did paleolithic people go after they left africa
AI:

> Finished chain.
Bot: Paleolithic humans spread across the globe as they migrated out of Africa approximately 60,000 years ago. They initially settled in the Middle East, Europe, and Asia, and later continued their expansion into Australia and the Americas. This migration was facilitated by advancements in technology, such as the development of sophisticated tools and weapons, which allowed them to adapt to various environments and ecosystems.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an 

In [16]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'simple', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Do you think time travel could ever be possible?
AI:

> Finished chain.
Bot: Time travel is a fascinating topic that has intrigued scientists, philosophers, and writers for centuries. While there are currently no proven methods or technologies that allow us to travel through time, various theories and hypotheses have been proposed. Some of these include wormholes, quantum mechanics, and theories involving multiple dimensions. However, these ideas remain speculative at this point, and it's impossible to definitively say if or when time travel will become a reality.


> Entering new ConversationChain chain...
Prompt after formatting:
Th

Please make a short report abount differences between used memory types

Report:

- Summary memory uses the LLM model to make a summary of the conversation.
- Window Size memory saves the last K messages
- Simple type of memory just saves all messages into the buffer

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric 

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [37]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor
import requests
from pydantic import Field

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

class WeatherResponse(BaseModel):
    weather: str = Field(description="Weather description")
    temperature: float = Field(description="Temperature in Celsius")

@tool
def get_wheather(city: str) -> WeatherResponse:
    """Get the current weather for a given city."""
    try:
        req = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric'
        response = requests.get(req)
        response.raise_for_status()
        data = response.json()
        
        weather = data['weather'][0]['description']
        temperature = data['main']['temp']
        
        return WeatherResponse(weather=weather, temperature=temperature)
    except requests.RequestException as e:
        raise ValueError(f"Failed to fetch weather data: {e}")
    except KeyError:
        raise ValueError("Invalid response from weather API.")

class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function

    def run(self, user_input: str):
        prompt = f"Определи город из запроса: '{user_input}' и верни только его название."
        model_answer = self.llm.invoke(prompt)
        city = model_answer.content

        weather_data = self.agent_function(city)

        prompt = (
            "Составь ответ о погоде для города на основе информации. Примеры: Город: Москва, Погода: ясно, Температура: 15°C\n"
            "Ответ: В Москве сейчас ясно, температура 15°C."
            f"Город: {city}\n"
            f"Погода: {weather_data.weather}\n"
            f"Температура: {weather_data.temperature}°C\n"
            "Ответ:"
        )
        model_answer = self.llm.invoke(prompt)
        output = model_answer.content.strip()

        return {'input': user_input, 'output': output}


Let's check it

In [38]:

giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_wheather)
user_input = "Какая погода сейчас в Ставрополе?"
openwheatertool.run(user_input)

{'input': 'Какая погода сейчас в Ставрополе?',
 'output': 'В Ставрополе сейчас облачно, температура 4.16°C.'}

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [23]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
import json
import datetime
from dateutil import parser
import os
from typing import Annotated

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)

@tool
def get_wheather(
        city: Annotated[str, "The destination city"]
) -> str:
    """Get the current weather for a given city."""
    try:
        req = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric'
        response = requests.get(req)
        response.raise_for_status()
        data = response.json()
        
        weather = data['weather'][0]['description']
        temperature = data['main']['temp']
        
        return f'Погода в {city}: {weather}. Температура: {temperature}'
    except requests.RequestException as e:
        raise ValueError(f"Failed to fetch weather data: {e}")
    except KeyError:
        raise ValueError("Invalid response from weather API.")



@tool
def get_url_booking_hotels(
    date_in_str: Annotated[str, "The check-in date for the hotel, formatted as a string (e.g., 'YYYY-MM-DD')"],
    date_out_str: Annotated[str, "The check-out date for the hotel, formatted as a string (e.g., 'YYYY-MM-DD')"],
    city: Annotated[str, "The destination city where the hotel is being booked"]
) -> str:
    """Find the hotel"""
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?' 
    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Вот Ваша ссылка для бронирования отеля: {url} в {city} на {date_in_str} / {date_out_str}'


@tool
def get_url_booking_tikets(
    city_from: Annotated[str, "The starting city or location of the trip"],
    city_to: Annotated[str, "The destination city or location of the trip"], 
    date_in_str: Annotated[str, "The departure date for the trip, formatted as a string (e.g., 'YYYY-MM-DD')"], 
    date_out_str: Annotated[str, "The return date for the trip, formatted as a string (e.g., 'YYYY-MM-DD')"]
) -> str:
    """Book the tickets"""
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?' 
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Вот Ваша ссылка для заказа билетов: {url} из {city_from} в {city_to} на {date_in_str} / {date_out_str}'



class MultiAgent:
    def __init__(self, llm, agent_function_wheater, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_function_wheater = agent_function_wheater
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets

        self.weather_agent = self._create_agent(
            tools=[self.agent_function_wheater],
            system_prompt="Ты агент, который помогает пользователям узнавать текущую погоду в различных городах."
        )
        self.hotels_agent = self._create_agent(
            tools=[self.agent_function_hotels],
            system_prompt="Ты агент, который помогает пользователям находить отели и бронировать их на заданные даты."
        )
        self.tickets_agent = self._create_agent(
            tools=[self.agent_function_tickets],
            system_prompt="Ты агент, который помогает пользователям бронировать авиабилеты на заданные направления и даты."
        )

        self.city_from = 'Москва' # hardcoded for this task


    def _create_agent(self, tools, system_prompt):
        agent = create_gigachat_functions_agent(self.llm, tools)
        return AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=False,
        )


    def run(self, user_input: str):
        # 1. Book the hotel
        hotel_response = self.hotels_agent.invoke({
            "input": (
                f"The user has requested to organize a trip with the following details: {user_input}. "
                f"Identify the destination city and dates mentioned by the user, and provide a booking link for hotels in that city "
                f"with check-in and check-out dates matching the specified travel dates."
                f"Example of the output: Вот Ваша ссылка для бронирования отеля: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2 в Казань на 06.10.2021 / 08.10.2021 "
            )
        })
        hotel_answer = hotel_response['output'] 

        # 2. Buy ticket
        ticket_response = self.tickets_agent.invoke({
            "input": (
                f"The user plans a trip with these details: {user_input}. "
                f"Find the departure city as '{self.city_from}' and determine the destination city and travel dates. "
                f"Provide a booking link for flights from {self.city_from} to the user's specified destination on the given dates."
                f"Example of the output: 'Вот Ваша ссылка для заказа билетов: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2 из Владивастока в Тайланд на на 06.10.2021 / 08.10.2021 "

            )
        })
        ticket_answer = ticket_response['output'] 

        # 3. Get the weather for given period
        weather_response = self.weather_agent.invoke({
            "input": (
                f"The user has mentioned a trip in the following details: {user_input}. "
                f"Identify the destination city for the trip and retrieve the current weather conditions for that location. "
                f"Provide a weather summary including temperature and overall conditions."
                f"Example of the output: в Москве сейчас пасмурно. Текущая температура составляет 3 градуса Цельсия."
            )
        })
        weather_answer = weather_response['output'] 

        return hotel_answer, ticket_answer, weather_answer


In [24]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_wheather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода"
answers = traveler.run(user_input)
for answer in answers:
    print(answer)

Вот Ваша ссылка для бронирования отеля: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2 в Санкт-Петербург на 2024-11-15 / 2024-11-25.
Вот Ваша ссылка для заказа билетов: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c213&klass=economy&oneway=2&return_date=2024-11-25&toId=c2&when=2024-11-15 из Москва в Санкт-Петербург на 2024-11-15 / 2024-11-25.
В Санкт-Петербурге сейчас ясно. Текущая температура составляет 12,81 градусов Цельсия.
